In [2]:
import numpy as np
import pandas as pd
import random

In [82]:
data={'item1':[3,5,3,2,2],'item2':[1,3,5,1,2],'item3':[5,1,4,2,4],'item4':[1,3,2,2,5],'item5':[5,5,4,5,3],'item6':[2,3,3,5,4]}
df=pd.DataFrame(data,index=['user1','user2','user3','user4','user5'])
df


,item1,item2,item3,item4,item5,item6
user1,3,1,5,1,5,2
user2,5,3,1,3,5,3
user3,3,5,4,2,4,3
user4,2,1,2,2,5,5
user5,2,2,4,5,3,4


In [83]:
t=np.array(df,dtype=float)
t

array([[3., 1., 5., 1., 5., 2.],
       [5., 3., 1., 3., 5., 3.],
       [3., 5., 4., 2., 4., 3.],
       [2., 1., 2., 2., 5., 5.],
       [2., 2., 4., 5., 3., 4.]])

In [84]:
miss=[]
idx=[]
for i in range(t.shape[0]):
    a=random.randint(0,5)
    miss.append(t[i][a])
    idx.append(a)
    t[i][a]=np.nan
print(miss)
print(idx)
    

[5.0, 3.0, 4.0, 5.0, 4.0]
[2, 1, 4, 5, 5]


In [85]:
t

array([[ 3.,  1., nan,  1.,  5.,  2.],
       [ 5., nan,  1.,  3.,  5.,  3.],
       [ 3.,  5.,  4.,  2., nan,  3.],
       [ 2.,  1.,  2.,  2.,  5., nan],
       [ 2.,  2.,  4.,  5.,  3., nan]])

In [86]:
rowmeans=(np.nanmean(t,axis=1,keepdims=True))
print(rowmeans)

[[2.4]
 [3.4]
 [3.4]
 [2.4]
 [3.2]]


In [87]:
t=t-rowmeans
t

array([[ 0.6, -1.4,  nan, -1.4,  2.6, -0.4],
       [ 1.6,  nan, -2.4, -0.4,  1.6, -0.4],
       [-0.4,  1.6,  0.6, -1.4,  nan, -0.4],
       [-0.4, -1.4, -0.4, -0.4,  2.6,  nan],
       [-1.2, -1.2,  0.8,  1.8, -0.2,  nan]])

In [88]:
for i in range(t.shape[0]):
    for j in range(t.shape[1]):
        if t[i][j]>0 or t[i][j]<0:
            continue
        else:
            t[i][j]=0
t

array([[ 0.6, -1.4,  0. , -1.4,  2.6, -0.4],
       [ 1.6,  0. , -2.4, -0.4,  1.6, -0.4],
       [-0.4,  1.6,  0.6, -1.4,  0. , -0.4],
       [-0.4, -1.4, -0.4, -0.4,  2.6,  0. ],
       [-1.2, -1.2,  0.8,  1.8, -0.2,  0. ]])

In [89]:
from numpy.linalg import norm

def cosine(a,b):
    cos=np.dot(a,b)/(norm(a)*norm(b))
    return cos

In [90]:
predictions=[]
for i in range(t.shape[0]):
    userdata=t[i][:]
    data1=t[0:i][:]
    data2=t[i+1:t.shape[0]][:]
    data=np.vstack((data1,data2))
    simi=[]
    idx=[]
    m=0
    for j in data:
        s=cosine(userdata,j)
        simi.append(s)
        idx.append(m)
        m+=1
    for j in range(len(simi)):
        for m in range(len(simi)-1):
            if simi[m]<=simi[j]:
                t1=simi[m]
                simi[m]=simi[j]
                simi[j]=t1

                t1=idx[m]
                idx[m]=idx[j]
                idx[j]=t1
    #let n=3
    n=3
    idx=idx[:n]
    simi=simi[:n]
    predata=data[idx]
    predindex=[]
    
    for j in range(len(userdata)):
        if userdata[j]==0:
            predindex.append(j)
            num=0
            den=0
            for k in range(len(predata)):
                num=num+simi[k]*predata[k][j]
                den=den+simi[k]
            userdata[j]=num/den
            predictions.append(userdata[j])
            
            





    

In [91]:
predictions=np.array(predictions)
predictions

array([-1.19866177, -1.92849446,  2.03436352, -0.4       , -0.4       ])

In [92]:
r=rowmeans.flatten()
r

array([2.4, 3.4, 3.4, 2.4, 3.2])

In [93]:
p=predictions+r
p

array([1.20133823, 1.47150554, 5.43436352, 2.        , 2.8       ])

In [94]:
miss

[5.0, 3.0, 4.0, 5.0, 4.0]

In [97]:
import math

MSE = np.square(np.subtract(miss,predictions)).mean()
rmse = math.sqrt(MSE)
rmse

4.797857689967558